In [ ]:
from collections import defaultdict
import os
import csv

import numpy as np
from matplotlib import pyplot as plt
import hoomd.md
import hoomd
import hoomd.deprecated
import freud
import gsd
import gsd.pygsd
import pybel
import mbuild as mb
import openbabel as ob

import ex_render
import utils

%matplotlib inline


# dictionary of SMARTS features using for coarse graining
features = utils.features_dict

# set to True if you want to view structures
vis = True

Building from the mbuild.Compound class allows us to more easily use already-developed tools! Anything that mbuild can import can be converted to a CG_Compound 

In [ ]:
smiles = "CCCCCCCCC"
mb_nonane = mb.load(smiles, smiles=True)

nonane = utils.CG_Compound()

# This from_mbuild function is very new and only copies the particles and bonds.
# I'd be interested to know where it breaks!
nonane.from_mbuild(mb_nonane)

nonane.visualize().show();

In [ ]:
# convert to pybel mol
mol = nonane.to_pybel()

# to_pybel imports all bonds as order=1, this will type the bond correctly 
# if the structure is good
mol.OBMol.PerceiveBondOrders()

cg_nonane = utils.coarse(mol, [features["alkyl_3"]], atomistic=True)

if vis:
    cg_nonane.visualize(color_scheme={"_A":"pink"}).show();

Run these if you want to step through examples of the CG_Compound coarse-graining

In [ ]:
!wget https://bitbucket.org/cmelab/msibi_tests/downloads/P3HT_4-density_0.75-n_compounds_20-traj.gsd

In [ ]:
gsdfile = "P3HT_4-density_0.75-n_compounds_20-traj.gsd"

# Coordinates are scaled from planckton sigma units
scale_factor = 0.356
comp0 = utils.CG_Compound.from_gsd(gsdfile, frame=0, scale=scale_factor)

# pybel will not correctly parse particles with AMBER typing
comp0.amber_to_element()

# unwrap feature won't move particles if the compound doesn't have bonds 
# that span the periodic boundary -- note the warning msg
comp0.unwrap()

# to_pybel is from mbuild PR555
mol0 = comp0.to_pybel()

# to_pybel imports all bonds as order=1, this will type the bond correctly 
# if the structure is good
mol0.OBMol.PerceiveBondOrders()

In [ ]:
# Notice that the initial frame is typed correctly
# the structure is good so pybel can type it
cg_comp0 = utils.coarse(mol0, [features["thiophene"],features["alkyl_3"]], atomistic=True)

if vis:
    cg_comp0.visualize(color_scheme={"Car":"black","_A":"pink","_B":"green"}).show();

In [ ]:
# same process as above but with last frame of trajectory
comp1 = utils.CG_Compound.from_gsd(gsdfile, frame=-1, scale=scale_factor)

comp1.amber_to_element()

# clearly this thing has some pbc problems
if vis:
    comp1.visualize(color_scheme={"Car":"black","_A":"pink","_B":"green"}).show();

In [ ]:
# fixed
comp1.unwrap()

if vis: 
    comp1.visualize(color_scheme={"Car":"black","_A":"pink","_B":"green"}).show();

In [ ]:
mol1 = comp1.to_pybel(box=mb.Box(comp1.box));
mol1.OBMol.PerceiveBondOrders();

# Even with fixing pbc issues, the last frame is distorted enough that 
# pybel can't recognise the features (bendy aromatic rings are NO)
cg_comp1 = utils.coarse(mol1,[features["thiophene"],features["alkyl_3"]],atomistic=True)

if vis:
    cg_comp1.visualize(color_scheme={"Car":"black","_A":"pink","_B":"green"}).show();

In [ ]:
# But since these are from the same trajectory, they have 
# the same number of particles in the same order, so we can
# "fix" the bad morphology using the good one!
mol1_fixed = utils.map_good_on_bad(mol0, mol1)

# Hey look it's fixed =D
cg_comp1_fixed = utils.coarse(mol1_fixed,[features["thiophene"],features["alkyl_3"]],atomistic=True)

if vis:
    cg_comp1_fixed.visualize(color_scheme={"Car":"black","_A":"pink","_B":"green"}).show();

# and we can rewrap it into the box
cg_comp1_fixed.wrap()

if vis:
    cg_comp1_fixed.visualize(color_scheme={"Car":"black","_A":"pink","_B":"green"}).show();

Run these if you want to do analysis (rdf, bond distribution) on the last frames of the trajectory

In [ ]:
!wget https://bitbucket.org/cmelab/msibi_tests/downloads/p3ht4-3_simple.gsd

In [ ]:
gsdfile = "p3ht4-3_simple.gsd"
scale_factor = 0.356
f = gsd.pygsd.GSDFile(open(gsdfile, 'rb'))
t = gsd.hoomd.HOOMDTrajectory(f)

# Get mol0
comp0 = utils.CG_Compound.from_gsd(gsdfile, frame=0, scale=scale_factor)
comp0.amber_to_element()
mol0 = comp0.to_pybel()
mol0.OBMol.PerceiveBondOrders()

lastframe = len(t)-1
firstframe = lastframe-10

quiet = True
vis = False

all_pairs = defaultdict(list)
all_bonds = defaultdict(list)
all_angles = defaultdict(list)

In [ ]:
# run analysis
for frame in range(firstframe, lastframe): 
    
    if not quiet: print(f"Coarse-graining frame {frame}...")
    
    comp = utils.CG_Compound.from_gsd(gsdfile, frame=frame, scale=scale_factor)
    comp.amber_to_element()
    comp.unwrap()
    mol = comp.to_pybel(box=mb.Box(comp.box))
    mol.OBMol.PerceiveBondOrders()
    test = mol.clone # I CANNOT EXPLAIN WHY THIS NEEDS TO BE HERE, BUT TRUST ME--IT DOES
    mol_fixed = utils.map_good_on_bad(mol0, mol)
    cg_comp_fixed = utils.coarse(mol_fixed, [features["thiophene"],features["alkyl_3"]])
    cg_comp_fixed.wrap()
    
    if vis:
        cg_comp_fixed.visualize(color_scheme={"Car":"black","_A":"pink","_B":"green"}).show();
    
    if not quiet: print("Done!\nStarting rdf...")
    
    if frame == firstframe:
        for tup in cg_comp_fixed.find_pairs():
            all_pairs[tup] = utils.get_compound_rdf(cg_comp_fixed, tup[0], tup[1])
    else:
        for tup in cg_comp_fixed.find_pairs():
            all_pairs[tup] = utils.get_compound_rdf(
                cg_comp_fixed,
                tup[0],
                tup[1],
                rdf = all_pairs[tup]
            )
        
    if not quiet: print("Done!\nStarting bond analysis...")
        
    particles = [p for p in cg_comp_fixed.particles()]
    bond_dict = cg_comp_fixed.find_bonds()
    for b_type in bond_dict:
        for pair in bond_dict[b_type]:
            if cg_comp_fixed.is_bad_bond(pair):
                pos0 = particles[pair[0]].pos
                pos1 = cg_comp_fixed.unwrap_position(pair)
            else:
                pos0 = particles[pair[0]].pos
                pos1 = particles[pair[1]].pos
            all_bonds[b_type].append(utils.distance(pos0,pos1))
            
    if not quiet: print("Done!\nStarting angle analysis...")
            
    angle_dict = cg_comp_fixed.find_angles()
    for a_type in angle_dict:
        for inds in angle_dict[a_type]:
            pair1 = (inds[0], inds[1])
            pair2 = (inds[1], inds[2])
            bad1 = cg_comp_fixed.is_bad_bond(pair1)
            bad2 = cg_comp_fixed.is_bad_bond(pair2)
            if bad1 and bad2:
                #the middle particle should be moved
                pos0 = particles[inds[0]].pos
                pos1 = cg_comp_fixed.unwrap_position(pair1)
                pos2 = particles[inds[2]].pos
                
            elif bad1:
                #the first particle should be moved
                pos0 = cg_comp_fixed.unwrap_position(pair1[::-1])
                pos1 = particles[inds[1]].pos
                pos2 = particles[inds[2]].pos
            
            elif bad2:
                #the last particle should be moved
                pos0 = particles[inds[0]].pos
                pos1 = particles[inds[1]].pos
                pos2 = cg_comp_fixed.unwrap_position(pair2)
            else:
                #all good
                pos0 = particles[inds[0]].pos
                pos1 = particles[inds[1]].pos
                pos2 = particles[inds[2]].pos
                
            all_angles[a_type].append(utils.get_angle(pos0,pos1,pos2))
            
    if not quiet: print("Done!")

In [ ]:
for pair in all_pairs:
    plt.plot(all_pairs[pair].R, all_pairs[pair].RDF)
    plt.title(f"rdf {pair[0]} {pair[1]}")
    plt.xlabel("r")
    plt.ylabel("g(r)")
    plt.show()

In [ ]:
for bond in all_bonds:
    start = min(all_bonds[bond])
    stop = max(all_bonds[bond])
    step = (stop-start)/30

    bins = [i for i in np.arange(start, stop, step)]
    bond_dist = np.empty([len(bins)-1,2])
    for i, length in enumerate(bins[1:]):
        in_bin = [b for b in all_bonds[bond] if b > bins[i] and b < bins[i+1]] 
        bond_dist[i,1] = len(in_bin)
        bond_dist[i,0] = np.mean((bins[i],bins[i+1]))
        
    plt.plot(bond_dist[:,0], bond_dist[:,1])
    plt.title(f"bond distribution {bond[0]}-{bond[1]}")
    plt.xlabel("bond length (nm)")
    plt.ylabel("# of bonds")
    plt.show()

In [ ]:
for angle in all_angles:
    start = min(all_angles[angle])
    stop = max(all_angles[angle])
    step = (stop-start)/20
    
    bins = [i for i in np.arange(start, stop, step)]
    angle_dist = np.empty([len(bins)-1,2])
    for i, length in enumerate(bins[1:]):
        in_bin = [a for a in all_angles[angle] if a > bins[i] and a < bins[i+1]] 
        angle_dist[i,1] = len(in_bin)
        angle_dist[i,0] = np.mean((bins[i],bins[i+1]))
        
    plt.plot(angle_dist[:,0], angle_dist[:,1])
    plt.title(f"angle distribution {angle[0]}-{angle[1]}-{angle[2]}")
    plt.xlabel("angle (rad)")
    plt.ylabel("# of bonds")
    plt.show()